<a href="https://colab.research.google.com/github/arssite/Datalysis/blob/main/DL_practiceSet_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: write a python program to implement a simple perceptron for binary classification.

import numpy as np

class Perceptron:
  def __init__(self, learning_rate, n_iters):
    self.lr = learning_rate
    self.n_iters = n_iters
    self.activation_func = self._unit_step_func
    self.weights = None
    self.bias = None

  def fit(self, X, y):
    n_samples, n_features = X.shape

    # Initialize weights and bias
    self.weights = np.zeros(n_features)
    self.bias = 0

    y_ = np.array([1 if i > 0 else 0 for i in y])

    for _ in range(self.n_iters):
      for i, x_i in enumerate(X):
        linear_output = np.dot(x_i, self.weights) + self.bias
        y_predicted = self.activation_func(linear_output)

        # Update weights and bias
        update = self.lr * (y_[i] - y_predicted)
        self.weights += update * x_i
        self.bias += update

  def predict(self, X):
    linear_output = np.dot(X, self.weights) + self.bias
    y_predicted = self.activation_func(linear_output)
    return y_predicted

  def _unit_step_func(self, x):
    return np.where(x >= 0, 1, 0)

# Example usage
X = np.array([[1, 2], [2, 3], [1, 1], [2, 2], [3, 1]])
y = np.array([1, 1, 0, 0, 0])

perceptron = Perceptron(learning_rate=0.1, n_iters=100)
perceptron.fit(X, y)

print("Weights:", perceptron.weights)
print("Bias:", perceptron.bias)

print("Predictions:", perceptron.predict(X))


Weights: [-0.1  0.1]
Bias: -0.1
Predictions: [1 1 0 0 0]


In [1]:
# prompt: function to perform forward propagation with one hidden layer

def forward_propagation_one_hidden_layer(X, W1, b1, W2, b2):
  """
  Performs forward propagation with one hidden layer.

  Args:
    X: Input data (n_samples, n_features)
    W1: Weights for the first layer (n_features, n_hidden)
    b1: Biases for the first layer (n_hidden,)
    W2: Weights for the second layer (n_hidden, n_classes)
    b2: Biases for the second layer (n_classes,)

  Returns:
    A2: Output of the second layer (n_samples, n_classes)
  """

  # Calculate the output of the first layer
  Z1 = np.dot(X, W1) + b1
  A1 = np.tanh(Z1)

  # Calculate the output of the second layer
  Z2 = np.dot(A1, W2) + b2
  A2 = 1 / (1 + np.exp(-Z2))

  return A2


In [ ]:
# prompt: backpropagation for neural network with one hidden layer

def backpropagation_one_hidden_layer(X, y, W1, b1, W2, b2, learning_rate):
  """
  Performs backpropagation for a neural network with one hidden layer.

  Args:
    X: Input data (n_samples, n_features)
    y: Target values (n_samples, n_classes)
    W1: Weights for the first layer (n_features, n_hidden)
    b1: Biases for the first layer (n_hidden,)
    W2: Weights for the second layer (n_hidden, n_classes)
    b2: Biases for the second layer (n_classes,)
    learning_rate: Learning rate

  Returns:
    Updated weights and biases
  """

  # Forward propagation
  A2 = forward_propagation_one_hidden_layer(X, W1, b1, W2, b2)

  # Calculate gradients
  dZ2 = A2 - y
  dW2 = np.dot(A1.T, dZ2) / X.shape[0]
  db2 = np.sum(dZ2, axis=0) / X.shape[0]

  dZ1 = np.dot(dZ2, W2.T) * (1 - np.power(A1, 2))
  dW1 = np.dot(X.T, dZ1) / X.shape[0]
  db1 = np.sum(dZ1, axis=0) / X.shape[0]

  # Update weights and biases
  W1 -= learning_rate * dW1
  b1 -= learning_rate * db1
  W2 -= learning_rate * dW2
  b2 -= learning_rate * db2

  return W1, b1, W2, b2


In [3]:
# prompt: implement Gradient Descent to minimize a aimple quadratic function

import numpy as np

def gradient_descent(f, df, x0, learning_rate=0.1, max_iter=100, tol=1e-6):
  """
  Performs gradient descent to minimize a function.

  Args:
    f: The function to minimize.
    df: The gradient of the function.
    x0: The initial guess for the minimum.
    learning_rate: The learning rate.
    max_iter: The maximum number of iterations.
    tol: The tolerance for convergence.

  Returns:
    x_min: The minimum of the function.
  """

  x = x0
  f_min = float('inf')

  for i in range(max_iter):
    gradient = df(x)
    x = x - learning_rate * gradient
    f_x = f(x)

    if f_x < f_min:
      f_min = f_x

    if np.abs(f_min - f_x) < tol:
      break

  return x

# Define the function and its gradient
def f(x):
  return x**2 + 5*x + 6

def df(x):
  return 2*x + 5

# Find the minimum
x_min = gradient_descent(f, df, x0=3.0)

print("Minimum found at:", x_min)
print("Minimum value:", f(x_min))


Minimum found at: 1.9
Minimum value: 19.11


In [ ]:
# prompt: implement a forward pass for simple cnn layer

import numpy as np

class Conv2D:
  def __init__(self, kernel_size, stride, padding):
    self.kernel_size = kernel_size
    self.stride = stride
    self.padding = padding
    self.weights = None
    self.bias = None

  def initialize_weights(self, input_channels, output_channels):
    self.weights = np.random.randn(output_channels, input_channels, self.kernel_size, self.kernel_size)
    self.bias = np.zeros(output_channels)

  def forward(self, X):
    # Add padding to the input
    X_padded = np.pad(X, ((0, 0), (0, 0), (self.padding, self.padding), (self.padding, self.padding)), mode='constant')

    # Calculate the output shape
    n_samples, n_channels, input_height, input_width = X.shape
    output_height = (input_height + 2 * self.padding - self.kernel_size) // self.stride + 1
    output_width = (input_width + 2 * self.padding - self.kernel_size) // self.stride + 1

    # Initialize the output
    Y = np.zeros((n_samples, self.weights.shape[0], output_height, output_width))

    # Perform the convolution
    for i in range(output_height):
      for j in range(output_width):
        for k in range(self.weights.shape[0]):
          for l in range(self.weights.shape[1]):
            Y[:, k, i, j] += np.sum(X_padded[:, l, i * self.stride:i * self.stride + self.kernel_size, j * self.stride:j * self.stride + self.kernel_size] * self.weights[k, l, :, :])

    # Add the bias
    Y += self.bias

    return Y

# Example usage
X = np.random.randn(10, 3, 32, 32)

conv_layer = Conv2D(kernel_size=3, stride=1, padding=1)
conv_layer.initialize_weights(3, 8)

Y = conv_layer.forward(X)

print("Input shape:", X.shape)
print("Output shape:", Y.shape)


In [4]:
# prompt: implement a simple RNN forward pass

def rnn_forward(X, W, U, b, h_prev):
  """
  Performs forward propagation for a simple RNN.

  Args:
    X: Input data (n_samples, n_timesteps, n_features)
    W: Weights for the current input (n_features, n_hidden)
    U: Weights for the previous hidden state (n_hidden, n_hidden)
    b: Biases (n_hidden,)
    h_prev: Previous hidden state (n_hidden,)

  Returns:
    h_next: Next hidden state (n_hidden,)
  """

  # Calculate the next hidden state
  h_next = np.tanh(np.dot(X, W) + np.dot(h_prev, U) + b)

  return h_next

# Example usage
X = np.random.randn(10, 5, 3)
W = np.random.randn(3, 4)
U = np.random.randn(4, 4)
b = np.random.randn(4)
h_prev = np.zeros(4)

h_next = rnn_forward(X, W, U, b, h_prev)

print("Input shape:", X.shape)
print("Hidden state shape:", h_next.shape)


Input shape: (10, 5, 3)
Hidden state shape: (10, 5, 4)
